In [ ]:
import os
import sys
from time import sleep

import GRBL
import numpy as np

import GCode

# Laser Material Calibration

Loop over laser power settings and feed rate to determine how a given material looks.

# Code:

In [6]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
sleep(1)
cnc.laser_mode = 1


ok


In [7]:
print(cnc.laser_mode)


1.0


In [8]:
cnc.status


'<Alarm|MPos:0.000,0.000,0.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [9]:
cnc.kill_alarm()


In [11]:
def laser_aim(self):
    self.cmd("G1F1")
    self.cmd("M3S1")
    input("Aim laser and press enter to continue...")
    self.cmd("M3S0")
    self.cmd("M5")


In [50]:
laser_aim(cnc)


Aim laser and press enter to continue...


In [20]:
def init():
    program = GCode.GCode()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    return program


In [21]:
def line(X=0, Y0=0, Yf=20, feed=100, power=255, dynamic_power=True):
    prog = GCode.GCode()
    prog.G0(X=X, Y=Y0)
    if dynamic_power:
        prog.M4(S=power)
    else:
        prog.M3(S=power)
    prog.G1(X=X, Y=Yf, F=feed)
    prog.M5()
    return prog


In [17]:
L = line()


In [22]:
L


<GCode>[cmds=4]

In [23]:
cnc.run(init())


0.3119051456451416

In [24]:
cnc.run(line())


0.4162750244140625

In [45]:
L0 = line(X=0, Y0=20, Yf=40, feed=200, power=255)
L1 = line(X=2, Y0=20, Yf=40, feed=200, power=255)


In [40]:
Xs = list(range(4, 142, 2))
powers = np.linspace(0, 255, len(Xs), dtype=np.uint8)


In [46]:
cnc.run(L0)
cnc.run(L1)

for X, power in zip(Xs, powers):
    L = line(X=X, Y0=20, Yf=40, feed=200, power=power)
    cnc.run(L)


In [47]:
Y0 = 40
Yf = Y0 + 20
feed = 300


In [48]:
L0 = line(X=0, Y0=Y0, Yf=Yf, feed=feed, power=255)
L1 = line(X=2, Y0=Y0, Yf=Yf, feed=feed, power=255)

cnc.run(L0)
cnc.run(L1)

for X, power in zip(Xs, powers):
    L = line(X=X, Y0=Y0, Yf=Yf, feed=feed, power=power)
    cnc.run(L)


In [49]:
cnc.cmd("G0X0Y0")


['ok', 'ok']